# Generate dollar bars
courtesy of towards data science

In [1]:
import pandas as pd
import numpy as np
import datetime
from IPython.display import clear_output

In [2]:
ans = input("Type 'yes, override BTCUSDT-dollar-bars-base.csv' to override existing dollar bars csv.")
if ans == 'yes, override BTCUSDT-dollar-bars-base.csv':
    df = pd.DataFrame([], columns=["time", "open", "high", "low", "close", "volume"])
    df.to_csv("../data/BTCUSDT-dollar-bars-base.csv", index=False)
else:
    pass

Type 'yes, override BTCUSDT-dollar-bars-base.csv' to override existing dollar bars csv. yes, override BTCUSDT-dollar-bars-base.csv


In [3]:
# Towards Data Science base
# expects a numpy array with trades
# each trade is composed of: [time, price, quantity]
def generate_dollarbars(trades, frequency=100000):
    times = trades[:,0]
    prices = trades[:,1]
    volumes = trades[:,2]
    ans = np.zeros(shape=(len(prices), 6))
    candle_counter = 0
    dollars = 0
    lasti = 0
    for i in range(len(prices)):
        dollars += volumes[i]*prices[i]
        if dollars >= frequency:
            ans[candle_counter][0] = times[i]                          # time
            ans[candle_counter][1] = prices[lasti]                     # open
            ans[candle_counter][2] = np.max(prices[lasti:i+1])         # high
            ans[candle_counter][3] = np.min(prices[lasti:i+1])         # low
            ans[candle_counter][4] = prices[i]                         # close
            ans[candle_counter][5] = np.sum(volumes[lasti:i+1])        # volume
            candle_counter += 1
            lasti = i+1
            dollars = 0
#             clear_output()
#             print(f"currently on index {i} out of {prices.shape[0]}")
    return ans[:candle_counter], trades[lasti:]

In [4]:
lot = None #lot is left over trades, trades at end of previous dataframe not used

for i in range(163):
    print(f"reading ../data/BTCUSDT-trades-{str(i)}.csv")
    df = pd.read_csv(f"../data/BTCUSDT-trades-{str(i)}.csv")
    print(f"converting to numpy...")
    df_np = df[["T", "p", "q"]].to_numpy()
    if type(lot) == np.ndarray:
        print(f"Adding previous trades to numpy array...")
        df_np = np.append(lot, df_np, 0) #Add left over trades to beginning of new dataframe
    print(f"Generating dollar bars...")
    ohlc, lot = generate_dollarbars(df_np)
    print(f"Reading, concating, saving csv...")
    ohlc = pd.DataFrame(ohlc, columns=["time", "open", "high", "low", "close", "volume"])
    pohlc = pd.read_csv("../data/BTCUSDT-dollar-bars-base.csv") #Previous ohlc
    ohlc = pd.concat([pohlc, ohlc])
    ohlc.to_csv("../data/BTCUSDT-dollar-bars-base.csv", index=False)
    ohlc = None
    pohlc = None
    clear_output()

## Remove duplicated index and set time index

In [52]:
df = pd.read_csv("../data/BTCUSDT-dollar-bars-base.csv")
len(df)

10517287

In [6]:
df["time"] = list(map(lambda x: datetime.datetime.fromtimestamp(x/1000), df.time))
df = df.set_index(["time"], drop=True)
df = df[~df.index.duplicated(keep='first')]
len(df)

10070717

In [7]:
df.to_csv("../data/BTCUSDT-dollar-bars.csv")